# Prompt Generation from User Requirements

https://github.com/langchain-ai/langgraph/blob/main/docs/docs/tutorials/chatbots/information-gather-prompting.ipynb

A **chat bot** that helps a user **generate a prompt**.
It will first collect requirements from the user, and then will generate the prompt (and refine it based on user input).
These are split into two separate states, and the LLM decides when to transition between them.

![information-gather-prompting](information-gather-prompting.png)


## Setup

First, let's install our required packages and set our OpenAI API key (the LLM we will use)

In [ ]:
%%capture --no-stderr
% pip install -U langgraph langchain_openai

## Gather information

First, let's define the part of the graph that will gather user requirements. This will be an LLM call with a specific system message. It will have access to a tool that it can call when it is ready to generate the prompt.

<div class="admonition note">
    <p class="admonition-title">Using Pydantic with LangChain</p>
    <p>
        This notebook uses Pydantic v2 <code>BaseModel</code>, which requires <code>langchain-core >= 0.3</code>. Using <code>langchain-core < 0.3</code> will result in errors due to mixing of Pydantic v1 and v2 <code>BaseModels</code>.
    </p>
</div>

In [26]:
from typing import List
import os
from langchain_core.messages import SystemMessage
from langchain_openai import ChatOpenAI

from pydantic import BaseModel

In [27]:
template = """Your job is to get information from a user about what type of prompt template they want to create.

You should get the following information from them:

- What the objective of the prompt is
- What variables will be passed into the prompt template
- Any constraints for what the output should NOT do
- Any requirements that the output MUST adhere to

If you are not able to discern this info, ask them to clarify! Do not attempt to wildly guess.

After you are able to discern all the information, call the relevant tool."""


def get_messages_info(messages):
    return [SystemMessage(content=template)] + messages
    
# Where is Berlin?
#   Args:
#     constraints: ["city"]
#     objective: locate
#     requirements: ["geographic location"]
#     variables: ["Berlin"]
class PromptInstructions(BaseModel):
    """Instructions on how to prompt the LLM."""

    objective: str
    variables: List[str]
    constraints: List[str]
    requirements: List[str]

os.environ["OPENAI_API_KEY"] = "NA"
llm = ChatOpenAI(
    model = "llama3.2",
    base_url = "http://localhost:11434/v1")

llm_with_tool = llm.bind_tools([PromptInstructions])


def info_chain(state):
    messages = get_messages_info(state["messages"])
    print("Gather Information input:", messages)
    response = llm_with_tool.invoke(messages)
    print("Gather Information output:", response)
    return {"messages": [response]}

## Generate Prompt

We now set up the state that will generate the prompt.
This will require a separate system message, as well as a function to filter out all message PRIOR to the tool invocation (as that is when the previous state decided it was time to generate the prompt

In [28]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage

# New system prompt
prompt_system = """Based on the following requirements, write a good prompt template:

{reqs}"""


# Function to get the messages for the prompt
# Will only get messages AFTER the tool call
def get_prompt_messages(messages: list):
    tool_call = None
    other_msgs = []
    for m in messages:
        if isinstance(m, AIMessage) and m.tool_calls:
            tool_call = m.tool_calls[0]["args"]
        elif isinstance(m, ToolMessage):
            continue
        elif tool_call is not None:
            other_msgs.append(m)
    print("tool_call:", tool_call)
    return [SystemMessage(content=prompt_system.format(reqs=tool_call))] + other_msgs


def prompt_gen_chain(state):
    messages = get_prompt_messages(state["messages"])
    print("Prompt Generation input:", messages)
    response = llm.invoke(messages)
    print("Prompt Generation output:", response)
    return {"messages": [response]}

## Define the state logic

This is the logic for what state the chatbot is in.
If the last message is a tool call, then we are in the state where the "prompt creator" (`prompt`) should respond.
Otherwise, if the last message is not a HumanMessage, then we know the human should respond next and so we are in the `END` state.
If the last message is a HumanMessage, then if there was a tool call previously we are in the `prompt` state.
Otherwise, we are in the "info gathering" (`info`) state.

In [29]:
from typing import Literal

from langgraph.graph import END


def get_state(state):
    messages = state["messages"]
    message=messages[-1]
    print("get_state message:", message)
    if isinstance(message, AIMessage) and message.tool_calls:
        print("get_state Goto addToolMessage")
        return "addToolMessage"
    elif not isinstance(message, HumanMessage):
        print("get_state Goto END")
        return END
    return "GatherInformation"

## Create the graph

We can now the create the graph.
We will use a SqliteSaver to persist conversation history.

In [30]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import StateGraph, START
from langgraph.graph.message import add_messages
from typing import Annotated
from typing_extensions import TypedDict


class State(TypedDict):
    messages: Annotated[list, add_messages]


memory = MemorySaver()
workflow = StateGraph(State)
workflow.add_node("GatherInformation", info_chain)
workflow.add_node("GeneratePrompt", prompt_gen_chain)


@workflow.add_node
def addToolMessage(state: State):
    message=state["messages"][-1]
    print("addToolMessage input:", message)
    tool_call_id = message.tool_calls[0]["id"]
    return {
        "messages": [
            ToolMessage(
                content="Prompt generated!",
                tool_call_id=tool_call_id,
            )
        ]
    }


workflow.add_conditional_edges("GatherInformation", get_state, ["addToolMessage", "GatherInformation", END])
workflow.add_edge("addToolMessage", "GeneratePrompt")
workflow.add_edge("GeneratePrompt", END)
workflow.add_edge(START, "GatherInformation")
graph = workflow.compile(checkpointer=memory)

In [ ]:
from IPython.display import Image, display

display(Image(graph.get_graph().draw_mermaid_png()))

## Use the graph

We can now use the created chatbot.

In [ ]:
import uuid

cached_human_responses = ["hi!", "rag prompt", "1 rag, 2 none, 3 no, 4 no", "red", "q"]
cached_response_index = 0
config = {"configurable": {"thread_id": str(uuid.uuid4())}}
while True:
    try:
        user = input("User (q/Q to quit): ")
    except:
        user = cached_human_responses[cached_response_index]
        cached_response_index += 1
    print(f"User (q/Q to quit): {user}")
    if user in {"q", "Q"}:
        print("AI: Byebye")
        break
    output = None
    for output in graph.stream(
        {"messages": [HumanMessage(content=user)]}, config=config, stream_mode="updates"
    ):
        last_message = next(iter(output.values()))["messages"][-1]
        last_message.pretty_print()

    if output and "prompt" in output:
        print("Done!")